<p style="font-size:20px;background-color:tomato;" >Dataset</p>
https://www.kaggle.com/datasets/gonzalorecioc/color-polygon-images

<p style="font-size:20px;"><strong>Importing Libraries</strong></p>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import skimage.color
import skimage.data
from tensorflow import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.regularizers import l1, l2
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

<p style="font-size:20px;"><strong>Importing Data</strong></p>
<p style="font-size:15px;">Creates train dataset from 10,000 images</p>

In [4]:
#Imports Data
train_ds = keras.utils.image_dataset_from_directory(
    directory='archive/images/content/images',
    labels=None,
    batch_size=32,
    image_size=(128, 128))

Found 10001 files belonging to 1 classes.


2024-01-23 09:41:08.176032: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4077 MB memory:  -> device: 0, name: Tesla K20m, pci bus id: 0000:24:00.0, compute capability: 3.5


<p style="font-size:20px;"><strong>Prints Pixels in Number Form</strong></p>
<p style="font-size:15px;">no bueno</p>

In [3]:
'''
#prints pixels for images
for element in train_ds:
  print(element)
'''

'\n#prints pixels for images\nfor element in train_ds:\n  print(element)\n'

<p style="font-size:20px;"><strong>Adjusts Collums</strong></p>
<p style="font-size:15px;">Drops majority of collums to make dataset easier to read, and prints dataset</p>

In [4]:
#Cleans up Data
df = pd.read_csv('archive/targets.csv')
df = df.drop(columns=['Unnamed: 0', 'bg_color', 'fg_color', 'bound_circle_x', 'bound_circle_y', 'bound_circle_r', 'rotation'])
df.head()

,filename,sides
0,70aaa621-1345-4541-a954-ee9856daaf18.png,4
1,ea0d8d14-596a-4365-b007-6325fc96e0f4.png,3
2,48c1fc5a-8b6b-4555-bb6a-a7d42a5cedc2.png,4
3,ad721906-604c-4682-ae5d-15539ef1dbc7.png,3
4,510dd5ea-fb92-432e-91cc-ed4654a03ebd.png,6


<p style="font-size:20px;"><strong>Adding Collums</strong></p>
<p style="font-size:15px;">Creates shape and copies sides data to it</p>

In [5]:
#Creates a new collum for shape
data = []
for i in range(len(df)):
  data.append(df.iloc[i].sides)
if data == 3:
    data = 'traingle'
df['shape'] = data
df.head()


,filename,sides,shape
0,70aaa621-1345-4541-a954-ee9856daaf18.png,4,4
1,ea0d8d14-596a-4365-b007-6325fc96e0f4.png,3,3
2,48c1fc5a-8b6b-4555-bb6a-a7d42a5cedc2.png,4,4
3,ad721906-604c-4682-ae5d-15539ef1dbc7.png,3,3
4,510dd5ea-fb92-432e-91cc-ed4654a03ebd.png,6,6


<p style="font-size:20px;"><strong>Creates Dictionaries</strong></p>
<p style="font-size:15px;">First dictionary to make code readable for humans</p>

In [6]:
#Dictionary to Changes sides into shapes
sides_to_names = {}
sides_to_names[3] = 'traingle'
sides_to_names[4] = 'square'
sides_to_names[5] = 'pentagon'
sides_to_names[6] = 'hexagon'
sides_to_names

{3: 'traingle', 4: 'square', 5: 'pentagon', 6: 'hexagon'}

<p style="font-size:15px;">Dictionary for indexing by model</p>

In [7]:
#Dictionary to change sides into encoding values
sides_to_encoding = {}
sides_to_encoding[3] = 0
sides_to_encoding[4] = 1
sides_to_encoding[5] = 2
sides_to_encoding[6] = 3
sides_to_encoding

{3: 0, 4: 1, 5: 2, 6: 3}

<p style="font-size:20px;"><strong>Creating shapes and encoding data</strong></p>
<p style="font-size:15px;">Using dictionaries to turn # of sides into data for each row<br> <br> </p>



<p style="font-size:20px;"><strong>Easy to understand code</strong></p>
<p style="color:gray;"> shapes = [] <br>
for i in range(3): <br>
  shape_name = sides_to_names[data[i]]<br>
  shapes.append(shape_name) </p>

In [8]:
#Turns sides into Shapes
shapes = [sides_to_names[sides] for sides in data]
encoding = [sides_to_encoding[sides] for sides in data]

'''
shapes = []
for i in range(3):
  shape_name = sides_to_names[data[i]]
  shapes.append(shape_name)

'''

df['shape'] = shapes
df['encoding'] = encoding
df.head

<bound method NDFrame.head of                                       filename  sides     shape  encoding
0     70aaa621-1345-4541-a954-ee9856daaf18.png      4    square         1
1     ea0d8d14-596a-4365-b007-6325fc96e0f4.png      3  traingle         0
2     48c1fc5a-8b6b-4555-bb6a-a7d42a5cedc2.png      4    square         1
3     ad721906-604c-4682-ae5d-15539ef1dbc7.png      3  traingle         0
4     510dd5ea-fb92-432e-91cc-ed4654a03ebd.png      6   hexagon         3
...                                        ...    ...       ...       ...
9995  e921970a-e09d-4760-ab46-9a33f863d1ef.png      5  pentagon         2
9996  bf398918-a475-42f1-adc0-59497120bcbd.png      3  traingle         0
9997  c4065a6d-5339-46f5-9917-2831698d4fb8.png      6   hexagon         3
9998  afd51b22-725a-4c43-8b40-dba5a83952c4.png      3  traingle         0
9999  e07de190-c136-465b-b2a9-04944aa7980c.png      4    square         1

[10000 rows x 4 columns]>

<p style="font-size:20px;"><strong>Creates data and lables</strong></p>
<p style="font-size:15px;">X is the question and y and is the answer<br>X is each a 128x128 image and y represents each polygon with an index</p>

In [9]:
y = df['encoding']

X = []
for img_file_name in df['filename']:
    img_path = "archive/images/content/images/" + img_file_name
    img = load_img(img_path, target_size=(128, 128))  # target size
    img_array = img_to_array(img)
    img_array /= 255.0  # normalize pixel values to range [0, 1]
    X.append(img_array)

X = np.array(X)

<p style="font-size:20px;"><strong>Splits data into train and test</strong></p>
<p style="font-size:15px;">Creates train, test, categorical, and validation varibles used to train and test the model</p>

In [10]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_validation, y_train_categorical, y_validation = train_test_split(
    X, y, test_size=0.2, random_state=42
)

<p style="font-size:20px;"><strong>Preventing Overtraining</strong></p>
<p style="font-size:15px;">Randomizing Pre-Model Data by randomly affecting attibtues of polygons<br>Randomizes rotation, width, height, zoom, and flip</p>

In [11]:
#Shifts Data to prevent overtaining
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')



<p style="font-size:20px;"><strong>Creates Extra Variables</strong></p>
<p style="font-size:15px;">Used for modified model fitting to implement data radomization and epoch validation</p>

In [12]:
#Creates Categorical Y data to implement data randomization
y_train_categorical = keras.utils.to_categorical(y_train, 4)
y_test_categorical = keras.utils.to_categorical(y_test, 4)

#y_train_categorical = to_categorical(y_train_categorical, num_classes=4)
y_validation = to_categorical(y_validation, num_classes=4)

<p style="font-size:20px;"><strong>Early Stopping</strong></p>
<p style="font-size:15px;">Ends training early once validation accuracy stops improving</p>

In [13]:
early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=4,  
    restore_best_weights=True  
)

<p style="font-size:20px;"><strong>Defining Model</strong></p>
<p style="font-size:15px;">Creates a two hidden layer CNN and compiling model using categorical crossentropy<br>
The goal is to reduce crossentropy loss, using categorical crossentropy reduces loss for multi-class models</p>

In [14]:
#Define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

<p style="font-size:20px;"><strong>Fitting Model</strong></p>
<p style="font-size:15px;">Fits model using randomized data (datagen) with a batch size of 32 and running 35 epochs</p>

In [15]:
# Fit Model
#model.fit(datagen.flow(X_train, y_train_categorical, batch_size=32), steps_per_epoch=len(X_train) / 32, epochs=20)

model.fit(datagen.flow(X_train, y_train_categorical, batch_size=32), 
          steps_per_epoch=len(X_train) / 32, 
          epochs=35, 
          validation_data=(X_validation, y_validation),
          callbacks=[early_stopping])

Epoch 1/35


2023-12-04 10:20:09.910476: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


250/250 [==============================] - 41s 132ms/step - loss: 1.3876 - accuracy: 0.2641 - val_loss: 1.3814 - val_accuracy: 0.2540
Epoch 2/35
250/250 [==============================] - 32s 128ms/step - loss: 1.3795 - accuracy: 0.2799 - val_loss: 1.3496 - val_accuracy: 0.2840
Epoch 3/35
250/250 [==============================] - 32s 128ms/step - loss: 1.3556 - accuracy: 0.3038 - val_loss: 1.3018 - val_accuracy: 0.3165
Epoch 4/35
250/250 [==============================] - 32s 128ms/step - loss: 1.1565 - accuracy: 0.4469 - val_loss: 0.7396 - val_accuracy: 0.7005
Epoch 5/35
250/250 [==============================] - 32s 128ms/step - loss: 0.7019 - accuracy: 0.7065 - val_loss: 0.3571 - val_accuracy: 0.8450
Epoch 6/35
250/250 [==============================] - 32s 128ms/step - loss: 0.4278 - accuracy: 0.8303 - val_loss: 0.1617 - val_accuracy: 0.9400
Epoch 7/35
250/250 [==============================] - 32s 127ms/step - loss: 0.2924 - accuracy: 0.8917 - val_loss: 0.1231 - val_accuracy: 0.9

<p style="font-size:20px;"><strong>Tests Model</strong></p>
<p style="font-size:15px;">Evaluates model loss and accuracy</p>

In [17]:
loss, accuracy = model.evaluate(X_test, y_test_categorical)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

63/63 [==============================] - 1s 20ms/step - loss: 0.0256 - accuracy: 0.9925
Test Loss: 0.02561393938958645
Test Accuracy: 0.9925000071525574


<p style="font-size:20px;background-color:SlateBlue;"><strong>Things that didn't work</strong></p>
<p style="font-size:15px;">Using SGD and RMS optimizers instead of ADAM<br><br>SGD provdes a fixe system for updating parameters and requires heavy tuning of hyperparameters<br>RMS is an adaptive learning model which updating parameters depending on recent gradiants<br>Adam is even more adaptive, tuning for each parameter and requires less tuning of hyperparameters<br><br>The reason ADAM works the best is because it does not need heavy hyperparameter tuning, unlike SGD and RMS</p>
<p style="font-size:18px;background-color:gray;"><strong>Overfitting</strong></p>
<p style="font-size:15px;">Originally the model was overfitting the data because the four classes of polygons were all similar placed in the origin, and only having different rotations<br>This issue was fixed by adding radomization to the data by using datagen/ImageDataGenerator. This augments the data to help the models generalization.</p>